# Lieferanten-Risikoanalyse

Dieses Notebook führt eine halb-automatisierte Risikoanalyse für Lieferantendaten durch. Führen Sie die Zellen nacheinander aus, indem Sie auf den "Play"-Button (▶) neben jeder Zelle klicken.

### Schritt 1: Konfiguration

In der folgenden Code-Zelle wird festgelegt, welche CSV-Datei analysiert werden soll. **Dies ist die einzige Stelle, die Sie anpassen müssen**, wenn Sie einen neuen Datensatz verwenden möchten. Ändern Sie einfach den Dateinamen innerhalb der Anführungszeichen.

In [ ]:
# Bibliotheken für Datenverarbeitung (pandas) und Grafiken (matplotlib) importieren
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches

# --- HIER DATEINAMEN ANPASSEN ---
csv_file_path = 'supplier_data.csv'
# --------------------------------

# Variable, um den Status der Datenprüfung zu speichern
data_is_correct = False

: 

### Schritt 2: Daten laden und bestätigen

Dieser Code liest die angegebene CSV-Datei ein und zeigt die ersten fünf Zeilen an. Anschließend werden Sie gefragt, ob die Daten korrekt aussehen. Antworten Sie mit 'ja', um die Analysen zu starten.

In [ ]:
try:
    df = pd.read_csv(csv_file_path)
    print(f"'{csv_file_path}' erfolgreich geladen. Hier sind die ersten 5 Zeilen zur Prüfung:")
    display(df.head()) # Zeigt eine formatierte Tabelle an
    
    # Interaktive Abfrage an den Benutzer
    user_input = input("Sehen diese Daten korrekt aus? (ja/nein): ")
    if user_input.lower() == 'ja':
        data_is_correct = True
        print("Daten bestätigt. Sie können nun mit den Analysen fortfahren.")
    else:
        data_is_correct = False
        print("Analyse abgebrochen. Bitte prüfen Sie die CSV-Datei oder den Dateinamen in Schritt 1.")
        
except FileNotFoundError:
    print(f"FEHLER: Die Datei '{csv_file_path}' wurde nicht gefunden. Bitte prüfen Sie den Dateinamen in Schritt 1.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

### Schritt 3: Analyse 1 - Risiko-Portfolio (Risiko vs. Umsatz)

Die folgende Zelle erstellt das Streudiagramm. Sie wird nur ausgeführt, wenn Sie in Schritt 2 mit 'ja' bestätigt haben.

In [ ]:
if data_is_correct:
    fig, ax = plt.subplots(figsize=(14, 9))
    ax.scatter(df['OverallRiskScore'], df['UmsatzvolumenEUR'], alpha=0.7, edgecolors='w', s=80)
    ax.set_title('Risiko-Portfolio-Analyse: Lieferantenrisiko vs. Umsatzvolumen', fontsize=16)
    ax.set_xlabel('Gesamtrisiko-Score (OverallRiskScore)', fontsize=12)
    ax.set_ylabel('Umsatzvolumen in EUR', fontsize=12)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.get_yaxis().set_major_formatter(mticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.xticks(range(int(df['OverallRiskScore'].min()) -1, int(df['OverallRiskScore'].max()) + 2))
    for i, txt in enumerate(df['SupplierID']):
        ax.annotate(txt, (df['OverallRiskScore'][i], df['UmsatzvolumenEUR'][i]),
                    textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
    median_risk = df['OverallRiskScore'].median()
    median_umsatz = df['UmsatzvolumenEUR'].median()
    ax.axvline(x=median_risk, color='grey', linestyle='--', lw=1.5)
    ax.axhline(y=median_umsatz, color='grey', linestyle='--', lw=1.5)
    plt.text(median_risk + 0.1, ax.get_ylim()[1], ' Hohes Risiko', color='red', va='top', ha='left')
    plt.text(median_risk - 0.1, ax.get_ylim()[1], ' Geringeres Risiko', color='green', va='top', ha='right')
    plt.show()
else:
    print("Analyse übersprungen, da die Daten nicht bestätigt wurden.")

### Schritt 4: Analyse 2 - Häufigste Risikoarten

Diese Zelle erstellt das Balkendiagramm und wird ebenfalls nur ausgeführt, wenn die Daten in Schritt 2 bestätigt wurden.

In [ ]:
if data_is_correct:
    risk_columns = [col for col in df.columns if col.endswith('Risk')]
    high_risk_counts = {}
    for col in risk_columns:
        count = df[col].value_counts().get('Hoch', 0)
        clean_name = col.replace('Risk', '')
        high_risk_counts[clean_name] = count
    risk_series = pd.Series(high_risk_counts).sort_values(ascending=True)
    social_risks = ['ChildLabor', 'ForcedLabor', 'WorkplaceSafety', 'FreedomOfAssociation', 'Discrimination', 'Wage', 'LandRights']
    ecological_risks = ['Pollution', 'WaterUsage', 'Deforestation']
    colors = ['royalblue' if risk in social_risks else 'forestgreen' if risk in ecological_risks else 'grey' for risk in risk_series.index]
    fig, ax = plt.subplots(figsize=(12, 8))
    risk_series.plot(kind='barh', ax=ax, color=colors)
    ax.set_title('Häufigkeit der Risikoart mit Bewertung "Hoch"', fontsize=16)
    ax.set_xlabel('Anzahl der Lieferanten mit Risiko "Hoch"', fontsize=12)
    ax.set_ylabel('Risikoart', fontsize=12)
    social_patch = mpatches.Patch(color='royalblue', label='Soziales Risiko')
    ecological_patch = mpatches.Patch(color='forestgreen', label='Ökologisches Risiko')
    ax.legend(handles=[social_patch, ecological_patch], loc='lower right')
    for index, value in enumerate(risk_series):
        ax.text(value, index, f' {value}', va='center')
    plt.tight_layout()
    plt.show()
else:
    print("Analyse übersprungen, da die Daten nicht bestätigt wurden.")